Ideas
- add population data per country
- NOTE: lat/long were added to daily data on march 1st: jan and feb tables only have province/country!


# Imports and functions

In [130]:
import glob
import pandas as pd
import os
import geopandas as gpd
import json
import chart_studio

import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly

In [11]:
# setting user, api key and access token
# The initialization step places a special .plotly/.credentials file in your home directory. 
# check using:
# !cat ~/.plotly/.credentials

chart_studio.tools.set_credentials_file(username='jd12006', api_key='GWFstLw9iUHo4yc1DJmq')
mapbox_access_token = 'pk.eyJ1IjoiamQxMjAwNiIsImEiOiJjazg0bjdldGQwc2F3M29xdDU2eGo5dnBzIn0.evQ7vR1JE6fVXLNEpmczvA'

In [50]:
!cat ~/.plotly/.credentials

{
    "username": "jd12006",
    "api_key": "GWFstLw9iUHo4yc1DJmq",
    "proxy_username": "",
    "proxy_password": "",
    "stream_ids": []
}

In [53]:
# set visibility
# public: Anyone can view. Private: Only you can view. Secret: anyone with a link can view.
chart_studio.tools.set_config_file(world_readable=False, sharing='secret')

# Data: COVID cases

## Time series
- one file per measure (confirmed dealths, recovered)
- one row per province/country
- no duplicates from latlongs or multiple entries in same day
- dates are columns, with new col added each day and zeros for missing data

In [232]:
# os.chdir('/Users/jdorni/Documents/training/COVID-19/csse_covid_19_data/csse_covid_19_time_series')
# deaths = pd.read_csv('time_series_19-covid-Deaths.csv')
# deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,19,22,22,27,29,29,29,33,35,40
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,2,2,2,3,4,10


## daily reports
- One file per day with latest results from certain countries.
- Contains all 3 measures: confirmed, deaths, recovered
- Countries dont send updated figures every day

In [151]:
def get_files(path):
    """
    List the csv files in a directory 
    """
    files = [x for x in os.listdir(path) if x.endswith(".csv")]
    return sorted(files)

path = '/Users/jdorni/Documents/training/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
files = get_files(path)
print(len(files))
print(files)

61
['01-22-2020.csv', '01-23-2020.csv', '01-24-2020.csv', '01-25-2020.csv', '01-26-2020.csv', '01-27-2020.csv', '01-28-2020.csv', '01-29-2020.csv', '01-30-2020.csv', '01-31-2020.csv', '02-01-2020.csv', '02-02-2020.csv', '02-03-2020.csv', '02-04-2020.csv', '02-05-2020.csv', '02-06-2020.csv', '02-07-2020.csv', '02-08-2020.csv', '02-09-2020.csv', '02-10-2020.csv', '02-11-2020.csv', '02-12-2020.csv', '02-13-2020.csv', '02-14-2020.csv', '02-15-2020.csv', '02-16-2020.csv', '02-17-2020.csv', '02-18-2020.csv', '02-19-2020.csv', '02-20-2020.csv', '02-21-2020.csv', '02-22-2020.csv', '02-23-2020.csv', '02-24-2020.csv', '02-25-2020.csv', '02-26-2020.csv', '02-27-2020.csv', '02-28-2020.csv', '02-29-2020.csv', '03-01-2020.csv', '03-02-2020.csv', '03-03-2020.csv', '03-04-2020.csv', '03-05-2020.csv', '03-06-2020.csv', '03-07-2020.csv', '03-08-2020.csv', '03-09-2020.csv', '03-10-2020.csv', '03-11-2020.csv', '03-12-2020.csv', '03-13-2020.csv', '03-14-2020.csv', '03-15-2020.csv', '03-16-2020.csv', '03-17

In [171]:
# get all CSVs 

# NOTE: lat/long were added to daily data on march 1st: jan and feb tables only have province/country!
# So need to load months separately


file_dict = {}
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
path = '/Users/jdorni/Documents/training/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'

for mon in months:
    try:
        file_path = f'{path}{mon}-*.csv'
        file_dict[mon] = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', file_path))))
    except Exception as e:
        break

01
02
03


In [196]:
# add NaNs for lat/long in jan/feb 
# (can join on country and province later if needed?)

df_janfeb = pd.concat([file_dict['01'], file_dict['02']]).drop_duplicates()
df_janfeb['Latitude'] = np.nan
df_janfeb['Longitude'] = np.nan

In [197]:
df_march_onwards = pd.concat([file_dict[k] for k in file_dict.keys() if k not in ['01', '02']])

In [199]:
df = pd.concat([df_janfeb, df_march_onwards])

In [200]:
# cleaning

len0 = len(df)
df = df.drop_duplicates(subset=None, keep='first', inplace=False)
print('dropped', len0 - len(df), 'duplicated rows')

# organise cols
df.rename({'Country/Region': 'country', 'Province/State': 'province'}, axis=1, inplace=True)
df.columns = [x.lower() for x in df.columns]
df.fillna('No data', inplace=True)

# fix timestamps
df['last_updated_ts'] = pd.to_datetime(df["last update"])
df['last_updated_d'] = df["last_updated_ts"].dt.date
#df.drop('last update', axis=1, inplace=True)
#df.drop(['last update', 'last_updated_ts'], axis=1, inplace=True)

# check incase 2 updates in a day
len0 = len(df)
df = df.drop_duplicates(subset=None, keep='first', inplace=False)
print('dropped', len0 - len(df), 'duplicated rows')

df.head()

dropped 1608 duplicated rows
dropped 0 duplicated rows


,province,country,last update,confirmed,deaths,recovered,latitude,longitude,last_updated_ts,last_updated_d
0,Hubei,Mainland China,1/31/2020 23:59,5806,204,141,No data,No data,2020-01-31 23:59:00,2020-01-31
1,Zhejiang,Mainland China,1/31/2020 23:59,538,No data,14,No data,No data,2020-01-31 23:59:00,2020-01-31
2,Guangdong,Mainland China,1/31/2020 23:59,436,No data,11,No data,No data,2020-01-31 23:59:00,2020-01-31
3,Henan,Mainland China,1/31/2020 23:59,352,2,3,No data,No data,2020-01-31 23:59:00,2020-01-31
4,Hunan,Mainland China,1/31/2020 23:59,332,No data,2,No data,No data,2020-01-31 23:59:00,2020-01-31


###### NOTE DATA NEED FURTHER CLEANING AS THERE ARE DUPLICATES WITHIN DAYS if updated twice in same day, or different date format, missing latlongs etc. But decided to move back to Bokeh


# Plotly

Data needs to be dict format

In [58]:
categories = ['confirmed', 'deaths', 'recovered']

In [ ]:
# make a data dict (IN PROG)

data = []
for cat in categories:
    data = dict(
            lat = df.loc[df['EVENT_TYPE'] == event,'BEGIN_LAT'],
            lon = df.loc[df['EVENT_TYPE'] == event,'BEGIN_LON'],
            name = event,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(event_data)
    